<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`: 
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [4]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell 
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()
        
    colunm_name = ' '.join(row.contents)
    
    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name    


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [6]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [14]:
# use requests.get() method with the provided static_url
# assign the response to a object
import requests

# URL for Falcon9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/Falcon_9"

# Send the GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    # Display the first 500 characters of the HTML page
    print(response.text[:500])
else:
    print("Failed to retrieve the page. Status code:", response.status_code)


Request successful!
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-


Create a `BeautifulSoup` object from the HTML `response`


In [17]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
import requests
from bs4 import BeautifulSoup

# URL for Falcon9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/Falcon_9"

# Send the GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    
    # Create a BeautifulSoup object and parse the HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Display the first 500 characters of the parsed HTML
    print(soup.prettify()[:500])  # This will print the formatted HTML
else:
    print("Failed to retrieve the page. Status code:", response.status_code)


Request successful!
<!DOCTYPE html>
<html class="client-nojs vector-feature-language-in-header-enabled vector-feature-language-in-main-page-header-disabled vector-feature-sticky-header-disabled vector-feature-page-tools-pinned-disabled vector-feature-toc-pinned-clientpref-1 vector-feature-main-menu-pinned-disabled vector-feature-limited-width-clientpref-1 vector-feature-limited-width-content-enabled vector-feature-custom-font-size-clientpref-1 vector-feature-appearance-pinned-clientpref-1 vector-feature-night-mode-


Print the page title to verify if the `BeautifulSoup` object was created properly 


In [19]:
# Use soup.title attribute
import requests
from bs4 import BeautifulSoup

# URL for Falcon9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/Falcon_9"

# Send the GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    
    # Create a BeautifulSoup object and parse the HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract the title of the page
    page_title = soup.title.string
    
    # Print the page title
    print("Page Title:", page_title)
else:
    print("Failed to retrieve the page. Status code:", response.status_code)


Request successful!
Page Title: Falcon 9 - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [21]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
import requests
from bs4 import BeautifulSoup

# URL for Falcon9 Launch Wikipedia page
url = "https://en.wikipedia.org/wiki/Falcon_9"

# Send the GET request
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    print("Request successful!")
    
    # Create a BeautifulSoup object and parse the HTML
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Extract the title of the page
    page_title = soup.title.string
    print("Page Title:", page_title)
    
    # Find all tables on the page
    tables = soup.find_all('table')
    
    # Loop through all found tables
    for i, table in enumerate(tables, start=1):
        print(f"\nTable {i}:")
        
        # Find all table headers (<th>) in the current table
        headers = table.find_all('th')
        
        # Extract and print the text from each header
        header_names = [header.get_text(strip=True) for header in headers]
        
        # Print the extracted column names
        if header_names:
            print("Column Names:", header_names)
        else:
            print("No column headers found in this table.")
else:
    print("Failed to retrieve the page. Status code:", response.status_code)


Request successful!
Page Title: Falcon 9 - Wikipedia

Table 1:
Column Names: ['Function', 'Manufacturer', 'Country of origin', 'Cost per launch', 'Size', 'Height', 'Diameter', 'Mass', 'Stages', 'Capacity', 'Payload toLEO', 'Orbital inclination', 'Mass', 'Payload toGTO', 'Orbital inclination', 'Mass', 'Payload toMars', 'Mass', '', 'Associated rockets', 'Based on', 'Derivative work', 'Launch history', 'Status', 'Launch sites', 'Total launches', 'Success(es)', 'Failure(s)', 'Partial failure(s)', 'Notable outcome(s)', 'Landings', 'First flight', 'Last flight', 'First stage', 'Height', 'Diameter', 'Powered by', 'Maximum thrust', 'Specific impulse', 'Burn time', 'Propellant', 'Second stage', 'Height', 'Diameter', 'Powered by', 'Maximum thrust', 'Specific impulse', 'Burn time', 'Propellant']

Table 2:
No column headers found in this table.

Table 3:
No column headers found in this table.

Table 4:
No column headers found in this table.

Table 5:
Column Names: ['Version', 'v1.0(retired)', 'v1.

Starting from the third table is our target table contains the actual launch records.


In [26]:
# Assuming 'tables' contains all the tables from the Wikipedia page
if len(tables) > 2:
    third_table = tables[2]
    print(third_table)
else:
    print("There are fewer than 3 tables on this page.")


<table class="wikitable">
<tbody><tr>
<td>Height
</td>
<td>41.2 m / 135.2 ft
</td></tr>
<tr>
<td>Height (with interstage)
</td>
<td>47.7 m / 156.5 ft
</td></tr>
<tr>
<td>Diameter
</td>
<td>3.7 m / 12 ft
</td></tr>
<tr>
<td>Empty Mass
</td>
<td>25,600 kg / 56,423 lb
</td></tr>
<tr>
<td>Propellant Mass
</td>
<td>395,700 kg/ 872,369 lb
</td></tr>
<tr>
<td>Structure Type
</td>
<td>LOX tank: <a href="/wiki/Monocoque" title="Monocoque">monocoque</a>
</td></tr>
<tr>
<td>
</td>
<td>Fuel tank: skin and stringer
</td></tr>
<tr>
<td>Structure Material
</td>
<td>Aluminum lithium skin; aluminum domes
</td></tr>
<tr>
<td>Landing Legs
</td>
<td>Number: 4
</td></tr>
<tr>
<td>
</td>
<td>Material: <a class="mw-redirect" href="/wiki/Carbon_fiber" title="Carbon fiber">carbon fiber</a>; <a class="mw-redirect" href="/wiki/Aluminum_honeycomb" title="Aluminum honeycomb">aluminum honeycomb</a>
</td></tr>
<tr>
<td>Number of Merlin Engines
</td>
<td>9 sea level
</td></tr>
<tr>
<td>Propellant
</td>
<td>LOX / RP-1

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [33]:
# Assuming 'tables' contains all the tables from the Wikipedia page
if len(tables) > 2:
    first_launch_table = tables[2]  # Define the third table

    column_names = []

    # Define a function to process the column header text
    def extract_column_from_header(header):
        return header.get_text(strip=True)  # Strip whitespace around the text

    # Find all <th> elements (headers) in the third table
    headers = first_launch_table.find_all('th')

    # Iterate over each header element, extract the column name, and append to column_names
    for header in headers:
        column_name = extract_column_from_header(header)
        if column_name is not None and len(column_name) > 0:  # Check if the column name is non-empty
            column_names.append(column_name)

    # Print the list of column names
    print(column_names)
else:
    print("There are fewer than 3 tables on this page.")


[]


Check the extracted column names


In [36]:
print(column_names)

[]


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [40]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

KeyError: 'Date and time ( )'

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [ ]:
extracted_row = 0
#Extract each table 
for table_number,table in enumerate(soup.find_all('table',"wikitable plainrowheaders collapsible")):
   # get table row 
    for rows in table.find_all("tr"):
        #check to see if first table heading is as number corresponding to launch a number 
        if rows.th:
            if rows.th.string:
                flight_number=rows.th.string.strip()
                flag=flight_number.isdigit()
        else:
            flag=False
        #get table element 
        row=rows.find_all('td')
        #if it is number save cells in a dictonary 
        if flag:
            extracted_row += 1
            # Flight Number value
            # TODO: Append the flight_number into launch_dict with key `Flight No.`
            #print(flight_number)
            datatimelist=date_time(row[0])
            
            # Date value
            # TODO: Append the date into launch_dict with key `Date`
            date = datatimelist[0].strip(',')
            #print(date)
            
            # Time value
            # TODO: Append the time into launch_dict with key `Time`
            time = datatimelist[1]
            #print(time)
              
            # Booster version
            # TODO: Append the bv into launch_dict with key `Version Booster`
            bv=booster_version(row[1])
            if not(bv):
                bv=row[1].a.string
            print(bv)
            
            # Launch Site
            # TODO: Append the bv into launch_dict with key `Launch Site`
            launch_site = row[2].a.string
            #print(launch_site)
            
            # Payload
            # TODO: Append the payload into launch_dict with key `Payload`
            payload = row[3].a.string
            #print(payload)
            
            # Payload Mass
            # TODO: Append the payload_mass into launch_dict with key `Payload mass`
            payload_mass = get_mass(row[4])
            #print(payload)
            
            # Orbit
            # TODO: Append the orbit into launch_dict with key `Orbit`
            orbit = row[5].a.string
            #print(orbit)
            
            # Customer
            # TODO: Append the customer into launch_dict with key `Customer`
            customer = row[6].a.string
            #print(customer)
            
            # Launch outcome
            # TODO: Append the launch_outcome into launch_dict with key `Launch outcome`
            launch_outcome = list(row[7].strings)[0]
            #print(launch_outcome)
            
            # Booster landing
            # TODO: Append the launch_outcome into launch_dict with key `Booster landing`
            booster_landing = landing_status(row[8])
            #print(booster_landing)
            

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [ ]:
df= pd.DataFrame({ key:pd.Series(value) for key, value in launch_dict.items() })

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab. 

Following labs will be using a provided dataset to make each lab independent. 


<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


<!--
## Change Log
-->


<!--
| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |
-->


Copyright © 2021 IBM Corporation. All rights reserved.
